In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D

import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data

from tqdm import tqdm
from sklearn import datasets
import random

In [3]:
import mylibrary.datasets as datasets

In [4]:
device = torch.device("cuda:0")
# device = torch.device("cuda:1")
# device = torch.device("cpu")

In [5]:
mnist = datasets.FashionMNIST()
# mnist.download_mnist()
# mnist.save_mnist()
train_data, train_label_, test_data, test_label_ = mnist.load()

train_data = train_data / 255.
test_data = test_data / 255.

train_size = len(train_label_)

In [6]:
## converting data to pytorch format
train_data = torch.Tensor(train_data)
test_data = torch.Tensor(test_data)
train_label = torch.LongTensor(train_label_)
test_label = torch.LongTensor(test_label_)

In [7]:
## converting data to pytorch format
train_data = torch.Tensor(train_data)
test_data = torch.Tensor(test_data)
train_label = torch.LongTensor(train_label_)

In [8]:
input_size = 784
output_size = 10

In [9]:
class MNIST_Dataset(data.Dataset):
    
    def __init__(self, data, label):
        self.data = data
        self.label = label
        
#         self.label = mask.type(torch.float32).reshape(-1,1)
        self._shuffle_data_()
        
    def __len__(self):
        return len(self.data)
    
    def _shuffle_data_(self):
        randidx = random.sample(range(len(self.data)), k=len(self.data))
        self.data = self.data[randidx]
        self.label = self.label[randidx]
    
    def __getitem__(self, idx):
        img, lbl = self.data[idx], self.label[idx]
        return img, lbl

In [10]:
train_dataset = MNIST_Dataset(train_data, train_label)
test_dataset = MNIST_Dataset(test_data, test_label)

In [11]:
learning_rate = 0.0003
batch_size = 50

In [12]:
train_loader = data.DataLoader(dataset=train_dataset, num_workers=4, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(dataset=test_dataset, num_workers=4, batch_size=batch_size, shuffle=False)

## Making of matrix factorized layer

In [13]:
class GroupLinear(nn.Module):
    def __init__(self, input_dim, output_dim, group_size=2, bias=True):
        super().__init__()
        assert input_dim >= group_size, \
                    f"Input dim:{input_dim} must be >= Group size: {group_size}"
        if output_dim < input_dim/group_size:
            print("Some inputs are ignored in the output")
            
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.group_size = group_size
        
        self.weight = torch.randn(group_size, 1).unsqueeze(0).repeat_interleave(output_dim, dim=0)
        self.weight = nn.Parameter(self.weight)
        self.bias = None
        if bias:
            self.bias = nn.Parameter(torch.zeros(output_dim))
            
        self.order = self.get_random_groups()
#         print(self.order)
#         print(np.unique(self.order, return_counts=True))
        
    def get_random_groups(self):
        rand_order = np.random.permutation(self.input_dim) ## all inputs are included
        if self.output_dim*self.group_size < self.input_dim:
            rand_order = rand_order[:self.output_dim*self.group_size]
#         print(rand_order)
        
        ord0 = []
        for i in range(self.output_dim):
            v = np.random.permutation(self.input_dim)[:self.group_size]
            ord0.append(v)
        ord0 = np.array(ord0)
        
#         print(ord0.shape)
#         print(ord0.T)
        
        ord0 = ord0.T.reshape(-1)
        ord0[:len(rand_order)] = rand_order
        ord0 = ord0.reshape(self.group_size, -1).T
#         print(ord0)
        
        return ord0.reshape(-1)

#     def get_random_groups(self):
#         if self.output_dim*self.group_size < self.input_dim:
#             return np.random.permutation(self.input_dim)[:self.output_dim*self.group_size]
        
#         rand_order = np.random.permutation(self.input_dim)
#         _temp = np.random.permutation(self.output_dim*self.group_size-input_dim)%self.input_dim
#         rand_order.
#         print(rand_order)
#         rand_order = rand_order.reshape(self.group_size, -1)
#         print(rand_order)
#         ### change the order if group contains same units
#         pass
        
    def forward(self, x):
        bs, gs = x.shape[0], self.group_size
        x = x[:, self.order]
        
        x = x.view(bs, -1, gs).transpose(0,1)
        x = torch.bmm(x, self.weight)
        x = x.squeeze(2).transpose(1,0)
        if self.bias is not None:
            x = x + self.bias
        return x

In [14]:
gl = GroupLinear(8, 8, group_size=2)

In [15]:
gl.weight

Parameter containing:
tensor([[[ 0.0194],
         [-0.5899]],

        [[ 0.0194],
         [-0.5899]],

        [[ 0.0194],
         [-0.5899]],

        [[ 0.0194],
         [-0.5899]],

        [[ 0.0194],
         [-0.5899]],

        [[ 0.0194],
         [-0.5899]],

        [[ 0.0194],
         [-0.5899]],

        [[ 0.0194],
         [-0.5899]]], requires_grad=True)

In [16]:
gl(torch.randn(3,8)).shape

torch.Size([3, 8])

In [18]:
class FactorNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.la1 = GroupLinear(784, 2048, group_size=10, bias=False)
        self.bn1 = nn.BatchNorm1d(2048)
        self.la2 = GroupLinear(2048, 2048, group_size=10, bias=False)
        self.bn2 = nn.BatchNorm1d(2048)
        self.la3 = GroupLinear(2048, 2048, group_size=10, bias=False)
        self.bn3 = nn.BatchNorm1d(2048)
        self.la4 = GroupLinear(2048, 784, group_size=10, bias=False)
        self.bn4 = nn.BatchNorm1d(784)
        self.fc = nn.Linear(784, 10)
        
    def forward(self, x):
        x = self.bn1(self.la1(x))
        x = torch.relu(x)
        x = self.bn2(self.la2(x))
        x = torch.relu(x)
        x = self.bn3(self.la3(x))
        x = torch.relu(x)
        x = self.bn4(self.la4(x))
        x = torch.relu(x)
        x = self.fc(x)
        return x

In [19]:
# class FactorNet(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.la1_a = GroupLinear(784, 2048, group_size=10, bias=False)
#         self.la1_b = GroupLinear(2048, 784, group_size=20, bias=False)
#         self.bn1 = nn.BatchNorm1d(784)

#         self.la2_a = GroupLinear(784, 2048, group_size=10, bias=False)
#         self.la2_b = GroupLinear(2048, 784, group_size=20, bias=False)
#         self.bn2 = nn.BatchNorm1d(784)
        
#         self.la3_a = GroupLinear(784, 2048, group_size=10, bias=False)
#         self.la3_b = GroupLinear(2048, 784, group_size=20, bias=False)
#         self.bn3 = nn.BatchNorm1d(784)
#         self.bn4 = nn.BatchNorm1d(784)
#         self.fc = nn.Linear(784, 10)
        
#     def forward(self, x):
#         x = self.bn1(self.la1_b(self.la1_a(x)))
#         x = torch.relu(x)
#         x = self.bn2(self.la2_b(self.la2_a(x)))
#         x = torch.relu(x)
#         x = self.bn3(self.la3_b(self.la3_a(x)))
#         x = torch.relu(x)
#         x = self.fc(x)
#         return x

In [20]:
class OrdinaryNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.la1 = nn.Linear(784, 784, bias=False)
        self.bn1 = nn.BatchNorm1d(784)
        self.la2 = nn.Linear(784, 784, bias=False)
        self.bn2 = nn.BatchNorm1d(784)
        self.la3 = nn.Linear(784, 10)
        
    def forward(self, x):
        x = self.bn1(self.la1(x))
        x = torch.relu(x)
        x = self.bn2(self.la2(x))
        x = torch.relu(x)
        x = self.la3(x)
        return x

In [21]:
model = FactorNet()
param_count = sum([torch.numel(p) for p in model.parameters()])
param_count

90986

In [22]:
model = OrdinaryNet()
param_count1 = sum([torch.numel(p) for p in model.parameters()])
param_count1, param_count1/param_count

(1240298, 13.631745543270393)

### Model Development

In [23]:
torch.manual_seed(0)
model = FactorNet().to(device)
# model = OrdinaryNet().to(device)
model

FactorNet(
  (la1): GroupLinear()
  (bn1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (la2): GroupLinear()
  (bn2): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (la3): GroupLinear()
  (bn3): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (la4): GroupLinear()
  (bn4): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=784, out_features=10, bias=True)
)

In [24]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)

In [25]:
print("number of params: ", sum(p.numel() for p in model.parameters()))

number of params:  90986


In [26]:
losses = []
train_accs = []
test_accs = []
EPOCHS = 20

for epoch in range(EPOCHS):
    
    train_acc = 0
    train_count = 0
    for xx, yy in tqdm(train_loader):
        xx, yy = xx.to(device), yy.to(device)

        yout = model(xx)
        loss = criterion(yout, yy)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(float(loss))

        outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
        correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
        train_acc += correct
        train_count += len(outputs)

    train_accs.append(float(train_acc)/train_count*100)
    train_acc = 0
    train_count = 0

    print(f'Epoch: {epoch},  Loss:{float(loss)}')
    test_count = 0
    test_acc = 0
    for xx, yy in tqdm(test_loader):
        xx, yy = xx.to(device), yy.to(device)
        with torch.no_grad():
            yout = model(xx)
        outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
        correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
        test_acc += correct
        test_count += len(xx)
    test_accs.append(float(test_acc)/test_count*100)
    print(f'Train Acc:{train_accs[-1]:.2f}%, Test Acc:{test_accs[-1]:.2f}%')
    print()

### after each class index is finished training
print(f'\t-> Train Acc {max(train_accs)} ; Test Acc {max(test_accs)}')

 22%|██▏       | 265/1200 [00:03<00:11, 80.70it/s]


KeyboardInterrupt: 

In [ ]:
### results are not that great